In [1]:
dados ='C:/Users/matto/Desktop/Leandro/Mestrado PPCIC/Dissertação/novo tema/Novo cj de dados/Conv1d/Dominio da Frequencia/Todos os arrays/dados_freq.npy'
rotulos ='C:/Users/matto/Desktop/Leandro/Mestrado PPCIC/Dissertação/novo tema/Novo cj de dados/rotulos.npy'

In [2]:
import numpy as np
X = np.load(dados)
y = np.load(rotulos)

In [3]:
from sklearn.model_selection import train_test_split

# Dividindo os dados em conjunto de treinamento e teste
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)



In [4]:
X_train.shape

(1493, 200, 1)

In [5]:
X_val.shape

(641, 200, 1)

In [6]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)

y_val = to_categorical(y_val)

In [7]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D
import optuna

def objective(trial):
    # Definindo o espaço de busca dos hiperparâmetros
    filter_size = trial.suggest_int('filter_size', 8, 600, log=True)
    kernel_size = trial.suggest_int('kernel_size', 2, 6)
    num_layers = trial.suggest_int('num_layers', 2, 4)
    dense_neurons = trial.suggest_int('dense_neurons', 60, 320, log=True)
    max_pool = 2
    dropout = trial.suggest_int('dropout', 2, 5, step=1) / 10.0
    learning_rate = trial.suggest_categorical('learning_rate', [0.0001, 0.0003,0.0006,0.001,0.003,0.006, 0.01])

    model = keras.Sequential()

    for i in range(num_layers):
        if i == 0:
            model.add(Conv1D(filters=filter_size, kernel_size=kernel_size, activation="relu", input_shape=(200, 1)))
            model.add(MaxPooling1D(pool_size=max_pool))
            model.add(Dropout(dropout))
        else:
            if filter_size < kernel_size:
                filter_size = kernel_size
            filter_size *= 2
            model.add(Conv1D(filters=filter_size, kernel_size=kernel_size, activation="relu"))
            model.add(MaxPooling1D(pool_size=max_pool))
            model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(dense_neurons, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    historico = model.fit(X_train, y_train, batch_size=32, epochs=25, validation_data=(X_val, y_val), verbose=1)

    return historico.history['val_accuracy'][-1]




In [8]:
import csv

# criação do objeto de estudo
study = optuna.create_study(direction="maximize")

# abertura do arquivo CSV em modo de escrita
with open("results.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(["trial", "accuracy", "hyperparameters"])  # cabeçalho

    # iteração dos trials
    
    study.optimize(objective, n_trials=80)
    best_trial = study.best_trial
    print("Trial {}: Accuracy = {}".format(i, best_trial.value))

    # escrita dos valores no arquivo CSV
    writer.writerow([i, best_trial.value, best_trial.params])

    print("Trial {}: Accuracy = {}".format(i, best_trial.value))

print(study.best_params)

[I 2023-07-07 10:42:40,361] A new study created in memory with name: no-name-c34de21a-fd4d-4e97-a47c-0503cff3e823


Epoch 1/25
47/47 [==============================] - 7s 126ms/step - loss: 1.4279 - accuracy: 0.8761 - val_loss: 0.1491 - val_accuracy: 0.9454
Epoch 2/25
47/47 [==============================] - 5s 116ms/step - loss: 0.0996 - accuracy: 0.9658 - val_loss: 0.0502 - val_accuracy: 0.9922
Epoch 3/25
47/47 [==============================] - 6s 118ms/step - loss: 0.0920 - accuracy: 0.9685 - val_loss: 0.0817 - val_accuracy: 0.9797
Epoch 4/25
47/47 [==============================] - 5s 115ms/step - loss: 0.0559 - accuracy: 0.9846 - val_loss: 0.0816 - val_accuracy: 0.9860
Epoch 5/25
47/47 [==============================] - 6s 121ms/step - loss: 0.0858 - accuracy: 0.9719 - val_loss: 0.0440 - val_accuracy: 0.9813
Epoch 6/25
47/47 [==============================] - 5s 117ms/step - loss: 0.0819 - accuracy: 0.9712 - val_loss: 0.0609 - val_accuracy: 0.9719
Epoch 7/25
47/47 [==============================] - 6s 117ms/step - loss: 0.0545 - accuracy: 0.9812 - val_loss: 0.0584 - val_accuracy: 0.9797
Epoch 

[I 2023-07-07 10:45:01,337] Trial 0 finished with value: 0.8564742803573608 and parameters: {'filter_size': 82, 'kernel_size': 4, 'num_layers': 4, 'dense_neurons': 156, 'dropout': 4, 'learning_rate': 0.006}. Best is trial 0 with value: 0.8564742803573608.


Epoch 1/25
47/47 [==============================] - 3s 52ms/step - loss: 0.4493 - accuracy: 0.9102 - val_loss: 0.0500 - val_accuracy: 0.9766
Epoch 2/25
47/47 [==============================] - 2s 48ms/step - loss: 0.0630 - accuracy: 0.9752 - val_loss: 0.0217 - val_accuracy: 0.9969
Epoch 3/25
47/47 [==============================] - 2s 51ms/step - loss: 0.0812 - accuracy: 0.9745 - val_loss: 0.0605 - val_accuracy: 0.9688
Epoch 4/25
47/47 [==============================] - 3s 55ms/step - loss: 0.0678 - accuracy: 0.9725 - val_loss: 0.0701 - val_accuracy: 0.9782
Epoch 5/25
47/47 [==============================] - 3s 55ms/step - loss: 0.0728 - accuracy: 0.9725 - val_loss: 0.0467 - val_accuracy: 0.9860
Epoch 6/25
47/47 [==============================] - 2s 53ms/step - loss: 0.0963 - accuracy: 0.9632 - val_loss: 0.0986 - val_accuracy: 0.9860
Epoch 7/25
47/47 [==============================] - 3s 58ms/step - loss: 0.0460 - accuracy: 0.9833 - val_loss: 0.0115 - val_accuracy: 0.9953
Epoch 8/25
47

[I 2023-07-07 10:46:08,118] Trial 1 finished with value: 0.9953197836875916 and parameters: {'filter_size': 81, 'kernel_size': 5, 'num_layers': 3, 'dense_neurons': 66, 'dropout': 2, 'learning_rate': 0.01}. Best is trial 1 with value: 0.9953197836875916.


...

Epoch 1/25
47/47 [==============================] - 3s 39ms/step - loss: 0.3614 - accuracy: 0.9049 - val_loss: 0.1434 - val_accuracy: 0.9594
Epoch 2/25
47/47 [==============================] - 2s 38ms/step - loss: 0.0421 - accuracy: 0.9859 - val_loss: 0.0327 - val_accuracy: 0.9875
Epoch 3/25
47/47 [==============================] - 2s 40ms/step - loss: 0.0217 - accuracy: 0.9920 - val_loss: 0.0245 - val_accuracy: 0.9922
Epoch 4/25
47/47 [==============================] - 2s 40ms/step - loss: 0.0372 - accuracy: 0.9893 - val_loss: 0.0127 - val_accuracy: 0.9953
Epoch 5/25
47/47 [==============================] - 2s 39ms/step - loss: 0.0184 - accuracy: 0.9933 - val_loss: 0.0136 - val_accuracy: 0.9938
Epoch 6/25
47/47 [==============================] - 2s 39ms/step - loss: 0.0244 - accuracy: 0.9926 - val_loss: 0.0113 - val_accuracy: 0.9953
Epoch 7/25
47/47 [==============================] - 2s 40ms/step - loss: 0.0304 - accuracy: 0.9893 - val_loss: 0.0176 - val_accuracy: 0.9938
Epoch 8/25
47

[I 2023-07-07 15:26:21,995] Trial 78 finished with value: 0.9906396269798279 and parameters: {'filter_size': 82, 'kernel_size': 3, 'num_layers': 2, 'dense_neurons': 93, 'dropout': 3, 'learning_rate': 0.0003}. Best is trial 39 with value: 1.0.


Epoch 1/25
47/47 [==============================] - 9s 173ms/step - loss: 0.9155 - accuracy: 0.9149 - val_loss: 0.0216 - val_accuracy: 0.9891
Epoch 2/25
47/47 [==============================] - 8s 170ms/step - loss: 0.0637 - accuracy: 0.9759 - val_loss: 0.0293 - val_accuracy: 0.9938
Epoch 3/25
47/47 [==============================] - 8s 171ms/step - loss: 0.0573 - accuracy: 0.9786 - val_loss: 0.0182 - val_accuracy: 0.9984
Epoch 4/25
47/47 [==============================] - 8s 175ms/step - loss: 0.0608 - accuracy: 0.9806 - val_loss: 0.0797 - val_accuracy: 0.9766
Epoch 5/25
47/47 [==============================] - 8s 174ms/step - loss: 0.0595 - accuracy: 0.9792 - val_loss: 0.0146 - val_accuracy: 0.9984
Epoch 6/25
47/47 [==============================] - 8s 172ms/step - loss: 0.0365 - accuracy: 0.9873 - val_loss: 0.0255 - val_accuracy: 0.9938
Epoch 7/25
47/47 [==============================] - 8s 175ms/step - loss: 0.0306 - accuracy: 0.9900 - val_loss: 0.0106 - val_accuracy: 0.9969
Epoch 

[I 2023-07-07 15:29:47,942] Trial 79 finished with value: 0.9953197836875916 and parameters: {'filter_size': 258, 'kernel_size': 4, 'num_layers': 2, 'dense_neurons': 102, 'dropout': 4, 'learning_rate': 0.006}. Best is trial 39 with value: 1.0.


NameError: name 'i' is not defined